In [258]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [259]:
seq_clean = pd.read_csv('seq_filter.csv')
test  = pd.read_json("/Users/Yannick/Desktop/Kaggle_covid/stanford-covid-vaccine/test.json",lines=True)
train = pd.read_json("/Users/Yannick/Desktop/Kaggle_covid/stanford-covid-vaccine/train.json",lines=True)

In [260]:
seq_clean = seq_clean.drop(columns = ['Unnamed: 0'])
seq_clean

,id,base,base_structure_type,base_predicted_loop_type,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_001f94081,0,0,0,0.1359,0.2613,0.2631,0.1501,0.2167,0.3297,0.7556,2.3375,0.3581,0.6382
1,id_001f94081,0,0,0,0.2070,0.3842,0.2860,0.2750,0.3475,1.5693,2.9830,3.5060,2.9683,3.4773
2,id_001f94081,1,0,0,0.1633,0.1372,0.0964,0.0947,0.1880,1.1227,0.2526,0.3008,0.2589,0.9988
3,id_001f94081,1,0,0,0.1452,0.2531,0.1574,0.1866,0.2124,0.8686,1.3789,1.0108,1.4552,1.3228
4,id_001f94081,1,0,0,0.1314,0.1798,0.1000,0.1369,0.1703,0.7217,0.6376,0.2635,0.7244,0.7877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108047,id_fff546103,2,2,1,0.0267,0.0626,0.0996,0.0638,0.0809,0.0272,0.5059,0.4998,0.2870,0.1820
108048,id_fff546103,3,2,1,0.0219,0.0411,0.0760,0.0520,0.0748,0.0382,0.2025,0.3057,0.2022,0.2038
108049,id_fff546103,1,2,1,0.0218,0.0391,0.0611,0.0520,0.0612,0.0381,0.1827,0.1642,0.2047,0.1120
108050,id_fff546103,0,2,1,0.0220,0.0509,0.0719,0.0595,0.0723,-0.0066,0.3261,0.1654,0.2447,0.1186


In [261]:
X_ = seq_clean[['base', 'base_structure_type', 'base_predicted_loop_type']]
model_full = LinearRegression()
y=seq_clean.iloc[:,-5:]
model_full.fit(X_, y)

LinearRegression()

In [262]:
regr.predict(X_test)

ValueError: could not convert string to float: 'id_00073f8be'

In [ ]:
regr.score(X_test, y_test)

In [ ]:
regr.feature_importances_

# import test file and transform for model 

In [ ]:
def expend_feature_test(df):
    """
    Return a dataframe with expension of sequence
    
    Args:
        df (Dataframe): same format as train
        i_row (int): number of row we want to expand
    
    Returns:
        sub_df: a dataframe with:
        number of rows = seq_scored
        columns name:[id, base, base_structure_type, base_predicted_loop_type, reactivity_error,
        deg_error_Mg_pH10,deg_error_pH10, deg_error_Mg_50C, deg_error_50C, reactivity, 
        deg_Mg_pH10, deg_pH10, deg_Mg_50C, deg_50C]
    """
    if (df.shape[0] != df.id.nunique()):
        print('repetition in RNA sequnence, clean dataframe first')
        return # dose the same as retunr None, which exit the function 
    
    else:
        col_names = ['id','base', 'base_structure_type', 'base_predicted_loop_type']

        #dataframe creation using list of lists
        # loop for each unique sequence
        sub_data = []
        for row_i in df.index:
            #loop for the legth of sequnece score (trian length is different from test)
            serie_i = df.loc[row_i] #panda series object
            seq_length  = serie_i['seq_length']
            for seq_i in range (seq_length):
                seq_data = [serie_i['id'] + '_' + str(seq_i), serie_i['sequence'][seq_i],
                            serie_i['structure'][seq_i], serie_i['predicted_loop_type'][seq_i]]
                sub_data.append(seq_data)

        sub_df = pd.DataFrame(sub_data, columns =col_names,  dtype = float) 
        return sub_df

In [ ]:
base_replace = {'G':0, 'A':1, 'C':2, 'U':3}
structure_replace ={'.':0, '(':1, ')':2}
loop_type_replace = {'E':0, 'S':1, 'H':2, 'B':3, 'X':4, 'I':5, 'M':6}

replace_types = [base_replace, structure_replace, loop_type_replace]

# vary the proportion of S/N data in the train test, use validation set to tune the 
def fianle_transform_test (dataframe, replace_type):
    # filter with SN_filter criteria 
    
    #use expend to change feature 
    data_filter_seq = expend_feature_test(dataframe)
    
    #make feature to integer
    for r_types in replace_type:  
        data_filter_seq = data_filter_seq.replace(r_types)
 
    return data_filter_seq

In [ ]:
X_test_ = fianle_transform_test(test, replace_types)
X_test_csv = X_test_csv[['base', 'base_structure_type', 'base_predicted_loop_type']]

In [ ]:
y_pred_ = model_full.predict(X_test_csv)

In [ ]:
y_pred_df_1 = pd.DataFrame(y_pred_, columns= ['reactivity', 
            'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C'])

y_pred_df_1.insert(0, "id_seqpos", X_test_['id'], True) 

In [ ]:
y_pred_df_1.describe()

In [ ]:
sum(y_pred_df_1['id_seqpos'] != sub.id_seqpos)

In [ ]:
y_pred_df_1.isnull().values.any()

In [ ]:
def expend_feature (df):
    """
    Return a dataframe with expension of sequence
    
    Args:
        df (Dataframe): same format as train
        i_row (int): number of row we want to expand
    
    Returns:
        sub_df: a dataframe with:
        number of rows = seq_scored
        columns name:[id, base, base_structure_type, base_predicted_loop_type, reactivity_error,
        deg_error_Mg_pH10,deg_error_pH10, deg_error_Mg_50C, deg_error_50C, reactivity, 
        deg_Mg_pH10, deg_pH10, deg_Mg_50C, deg_50C]
    """
    if (df.shape[0] != df.id.nunique()):
        print('repetition in RNA sequnence, clean dataframe first')
        return # dose the same as retunr None, which exit the function 
    
    else:
        col_names = ['id','base', 'base_structure_type', 'base_predicted_loop_type', 'reactivity_error',
            'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 
            'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

        #dataframe creation using list of lists
        # loop for each unique sequence
        sub_data = []
        for row_i in df.index:
            #loop for the legth of sequnece score (trian length is different from test)
            serie_i = df.loc[row_i] #panda series object
            seq_length  = serie_i['seq_scored']
            for seq_i in range (seq_length):
                seq_data = [serie_i['id'], serie_i['sequence'][seq_i],
                            serie_i['structure'][seq_i], serie_i['predicted_loop_type'][seq_i],
                            serie_i['reactivity_error'][seq_i], serie_i['deg_error_Mg_pH10'][seq_i],
                            serie_i['deg_error_pH10'][seq_i], serie_i['deg_error_Mg_50C'][seq_i],
                            serie_i['deg_error_50C'][seq_i], serie_i['reactivity'][seq_i],
                            serie_i['deg_Mg_pH10'][seq_i], serie_i['deg_pH10'][seq_i],
                            serie_i['deg_Mg_50C'][seq_i], serie_i['deg_50C'][seq_i]]
                sub_data.append(seq_data)
        sub_df = pd.DataFrame(sub_data, columns =col_names,  dtype = float) 
        return sub_df
base_replace = {'G':0, 'A':1, 'C':2, 'U':3}
structure_replace ={'.':0, '(':1, ')':2}
loop_type_replace = {'E':0, 'S':1, 'H':2, 'B':3, 'X':4, 'I':5, 'M':6}

replace_types = [base_replace, structure_replace, loop_type_replace]

# vary the proportion of S/N data in the train test, use validation set to tune the 
def fianle_transform_without_SN (dataframe, replace_type):
    # filter with SN_filter criteria 

    
    #use expend to change feature 
    data_filter_seq = expend_feature(dataframe)
    
    #make feature to integer
    for r_types in replace_type:  
        data_filter_seq = data_filter_seq.replace(r_types)
 
    return data_filter_seq

In [ ]:
seq_= fianle_transform_without_SN(train, replace_types)

In [ ]:
seq_

In [ ]:
X_ = seq_clean[['base', 'base_structure_type', 'base_predicted_loop_type']]
model_SN = LinearRegression()
y = seq_clean.iloc[:,-5:]
model_SN.fit(X_, y)

In [ ]:
y_pred_SN = model_SN.predict(X_test_csv)

In [ ]:
y_pred_df_SN = pd.DataFrame(y_pred_SN, columns= ['reactivity', 
            'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C'])

y_pred_df_SN.insert(0, "id_seqpos", X_test_['id'], True) 

In [ ]:
y_pred_df_SN.to_csv('sub_without_SN.csv', index=False)

In [ ]:
model_SN.coef_

In [ ]:
model_full.coef_

In [ ]:
y_pred_df_SN